In [71]:
import numpy as np

class FCLayer:
    def __init__(self, input_size, output_size):
        self.w = np.random.random((output_size, input_size)) - 0.5
        self.b = np.zeros((output_size, 1))
        self.layer_input = None

    def forward(self, layer_input):
        self.layer_input = layer_input
        return np.dot(self.w, layer_input) + self.b

    def backprop(self, layer_error, learning_rate=0.01):
        m = layer_error.shape[1]
        self.w -= np.dot(layer_error, self.layer_input.T) / m * learning_rate
        self.b -= np.sum(layer_error, axis=1, keepdims=True) / m * learning_rate
        return np.dot(self.w.T, layer_error)

class ActivationLayer:
    def __init__(self, func, func_prime):
        self.func = func
        self.func_prime = func_prime
        self.layer_input = None

    def forward(self, layer_input):
        self.layer_input = layer_input
        return self.func(layer_input)

    def backprop(self, layer_error, learning_rate=0.01):
        return self.func_prime(self.layer_input) * layer_error

class NeuralNetwork:
    def __init__(self, layers, loss, loss_prime, learning_rate=0.01):
        self.layers = layers
        self.loss = loss
        self.loss_prime = loss_prime
        self.learning_rate = learning_rate

    def forward(self, x):
        res = [x]
        for layer in self.layers:
            res.append(layer.forward(res[-1]))
        return res[-1]

    def fit(self, x, y, epochs=100, verbose=100):
        for i in range(epochs):
            ret = self.forward(x)
            loss = self.loss(ret, y)
            res = [self.loss_prime(ret, y)]
            for layer in reversed(self.layers):
                res.append(layer.backprop(res[-1], learning_rate=self.learning_rate))
            if i % verbose == 0:
                print(f'Loss at epoch {i}: {loss}')

In [72]:
relu = lambda x: np.maximum(0, x)
drelu = lambda x: 1.0 * (x > 0)

sigmoid = lambda x: 1 / (1 + np.exp(-x))
dsigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x))


nn = NeuralNetwork(
    layers=[
        FCLayer(2, 5),
        ActivationLayer(relu, drelu),
        FCLayer(5, 1),
        ActivationLayer(sigmoid, dsigmoid)
    ],
    loss=lambda p, y: np.sum((p - y) ** 2) / p.shape[1],
    loss_prime=lambda p, y: 2 * (p - y) / p.shape[1],
    learning_rate=0.1
)

In [73]:
x_train = np.asarray([0, 0, 1, 1, 0, 1, 0, 1]).reshape((2, 4))
y_train = np.asarray([0, 1, 1, 0])
nn.fit(x_train, y_train, epochs=10000, verbose=5000)

Loss at epoch 0: 0.23993651737074878
Loss at epoch 5000: 0.010232838719829846


In [74]:
nn.forward(x_train)

array([[0.07079301, 0.95947343, 0.95942056, 0.07079326]])